In [ ]:
import cv2
import torch
import numpy as np
import mediapipe as mp
import torch.nn as nn
from torchvision import transforms
from torchvision import models
from torch.nn.functional import normalize
import pandas as pd
import os

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Cargar backbone
backbone = models.resnet50(pretrained=False)
in_features = backbone.fc.in_features
backbone.fc = nn.Identity()
checkpoint = torch.load("../models/arcface_backbone.pth")
backbone.load_state_dict(checkpoint["backbone"])

embedding_layer = torch.nn.Linear(in_features, 512)
embedding_layer.load_state_dict(checkpoint["embedding"])

backbone = backbone.to(DEVICE).eval()
embedding_layer = embedding_layer.to(DEVICE).eval()

# Transform
transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((112,112)),
    transforms.ToTensor(),
    transforms.Normalize([0.5]*3, [0.5]*3)
])

# Galería de embeddings
reference_db = np.load("../models/gallery_embeddings.npy", allow_pickle=True).item()

# Mediapipe detection
mp_face_detection = mp.solutions.face_detection
face_detector = mp_face_detection.FaceDetection(model_selection=1, min_detection_confidence=0.9)

def get_embedding(face_img):
    face_tensor = transform(face_img).unsqueeze(0).to(DEVICE)
    with torch.no_grad():
        features = backbone(face_tensor)
        emb = embedding_layer(features)
        emb = normalize(emb, dim=1)
    return emb.squeeze(0).cpu().numpy()

def recognize_face(embedding, threshold=0.91):
    best_match = None
    best_score = -1
    for name, ref_emb in reference_db.items():
        score = np.dot(embedding, ref_emb)
        if score > best_score:
            best_score = score
            best_match = name
    if best_score >= threshold:
        return best_match
    return "Desconocido"

# Cambiar aquí
cap = cv2.VideoCapture(0)  # Webcam
# cap = cv2.VideoCapture("../data/crudo/Abir1.mp4")  # Video

while True:
    ret, frame = cap.read()
    if not ret:
        break

    rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = face_detector.process(rgb)

    if results.detections:
        for det in results.detections:
            bbox = det.location_data.relative_bounding_box
            ih, iw, _ = frame.shape
            x1 = max(int(bbox.xmin * iw), 0)
            y1 = max(int(bbox.ymin * ih), 0)
            w = int(bbox.width * iw)
            h = int(bbox.height * ih)
            x2 = min(x1 + w, iw)
            y2 = min(y1 + h, ih)

            face_img = frame[y1:y2, x1:x2]
            if face_img.size == 0:
                continue

            # Reconocimiento
            emb = get_embedding(face_img)
            name = recognize_face(emb)

            # Bounding box
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0,255,0), 2)
            cv2.putText(frame, name, (x1, y1-10), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255,255,255), 2)

            # Dibujar keypoints
            for kp in det.location_data.relative_keypoints:
                kp_x = int(kp.x * iw)
                kp_y = int(kp.y * ih)
                cv2.circle(frame, (kp_x, kp_y), 2, (0, 250, 0), -1)

    cv2.imshow("Recognition", frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

# === Procesar CSV al final ===
csv_file = "../results/asistencia.csv"   # Cambia esta ruta si tu CSV está en otro lado

# Verificar si el archivo existe
if not os.path.exists(csv_file):
    # Crear un DataFrame vacío con columnas esperadas
    df_empty = pd.DataFrame(columns=["Nombre", "Fecha"])
    df_empty.to_csv(csv_file, index=False)
    print("Archivo 'asistencia.csv' creado vacío.")

# Leer el archivo CSV
df = pd.read_csv(csv_file)

# Convertir columna de fecha a datetime
df['Fecha'] = pd.to_datetime(df['Fecha'])

# Ordenar por Nombre y Fecha
df = df.sort_values(by=['Nombre', 'Fecha'])

# Crear DataFrame vacío para resultados
df_processed = pd.DataFrame(columns=df.columns)

# Procesar cada persona
for nombre in df['Nombre'].unique():
    df_nombre = df[df['Nombre'] == nombre]
    primera = df_nombre.iloc[0]
    ultima = df_nombre.iloc[-1]

    # Agregar la primera fecha
    df_processed = pd.concat([df_processed, pd.DataFrame([primera])])

    # Agregar la última si es diferente
    if not primera.equals(ultima):
        df_processed = pd.concat([df_processed, pd.DataFrame([ultima])])

# Guardar el CSV con los resultados
df_processed.to_csv(csv_file, index=False)

print("Archivo CSV procesado: primeras y últimas fechas por persona.")

INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
W0000 00:00:1751315649.426786   13222 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
